In [1]:
import cmip6_cv.PrePARE.PrePARE as pp
import subprocess
import pandas as pd
import git 
from shutil import copy2

## 1. Settings. 

In [2]:
trunk="/mnt/lustre02/work/ik1017/CMIP6/data/"
prepareSetting = {
    "exec" : pp.__file__ ,
    "logChunk":"/mnt/lustre02/work/ik1017/CMIP6/meta/c34g-qc-prepare/logs/",
    "cmip6-cmor-table-path" : "/home/dkrz/k204210/cmip6-cmor-tables/Tables" }
dsetsinp = "/home/dkrz/k204210/catalog-to-manifest/catalogs/wg1subset-r1-datasets-pids-clean.csv.gz"
recentCV = "/home/dkrz/k204210/CMIP6-CV-20200617.json"

## 2. Initialize

In [3]:
dsetslist = pd.read_csv(dsetsinp)
g = git.Git(prepareSetting["cmip6-cmor-table-path"]) 
g.reset("--hard")
g.checkout("master")
copy2(prepareSetting["cmip6-cmor-table-path"]+"/CMIP6_CV.json", recentCV)

'/home/dkrz/k204210/CMIP6-CV-20200617.json'

Convert dataset to path to data in order to apply PrePARE on it

In [4]:
def filepathFromDset(dset) :
    return trunk+dset["dataset_id"].replace('.','/')

In [5]:
dsetslist["filepath"]=dsetslist.apply(lambda row: filepathFromDset(row), axis=1)

## 3. Prepare PrePARE

For applying PrePARE, we need the correct cmip6-cmor-tables release that matches the data_specs_version in the file.

In [6]:
tags = reversed(g.tag("-n").split("\n"))
taglist = pd.DataFrame(columns=["data_specs_version","tag_label","description"])
for tag in tags :
    tl = tag.split(" ", 1)[0]
    tllen = len(tl.split("."))
    if tllen > 3 :
        continue
    dsvnumber = tl.split(".")[tllen-1]
    dsvnumber = "".join(filter(str.isdigit, dsvnumber))
    dsv = "['01.00."+dsvnumber+"']"
    if taglist[taglist["data_specs_version"] == dsv].empty :
        taglist = taglist.append({"data_specs_version":dsv,
                                  "tag_label":tl,
                                  "description":tag.split(" ",1)[1]},
                                 ignore_index=True)

In [7]:
print(taglist)

   data_specs_version     tag_label  \
0        ['01.00.32']        6.9.32   
1        ['01.00.31']        6.8.31   
2        ['01.00.30']        6.6.30   
3        ['01.00.29']        6.5.29   
4        ['01.00.28']        6.4.28   
5        ['01.00.27']  6.3.27-fixed   
6        ['01.00.24']        6.2.24   
7        ['01.00.23']        6.1.23   
8        ['01.00.22']        6.1.22   
9        ['01.00.21']        6.0.21   
10       ['01.00.20']        6.0.20   
11       ['01.00.19']       6.0.19a   
12       ['01.00.18']        6.0.18   
13       ['01.00.17']        6.0.17   
14       ['01.00.16']        6.0.16   
15       ['01.00.15']        6.0.15   
16       ['01.00.14']        6.0.14   
17       ['01.00.13']        6.0.13   
18       ['01.00.12']        6.0.12   
19       ['01.00.11']        6.0.11   

                                          description  
0            Merge pull request #311 from PCMDI/01...  
1            Merge pull request #255 from PCMDI/cm...  
2           

## 4. Apply PrePARE

In [11]:
def checkSubset(tocheck):
    for index, row in tocheck.iterrows():
        tag2checkout = taglist[taglist["data_specs_version"]==row["data_specs_version"]]["tag_label"].to_list()
        g.reset("--hard")
        g.checkout(tag2checkout)
        copy2(recentCV, prepareSetting["cmip6-cmor-table-path"]+"/CMIP6_CV.json")
        #
        logPath=prepareSetting["logChunk"]+row["data_specs_version"].split('.')[2].split("'")[0]+"/"+row["dataset_id"]
        a = subprocess.run([prepareSetting["exec"],
                        "-l",logPath,
                        "--table-path",prepareSetting["cmip6-cmor-table-path"],
                        row["filepath"]], capture_output=True)

In [12]:
for dsv in dsetslist["data_specs_version"].unique().tolist() :
    tocheck = dsetslist[dsetslist["data_specs_version"] == dsv]
    checkSubset(tocheck)